In [ ]:
import pandas as pd

def display_csv_contents(file_path):
    try:
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)

        # Display the contents of the DataFrame
        print("CSV File Contents:")
        print(df)

    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Specify the path to your CSV file
csv_file_path = "teams.csv"

# Call the function to display the contents
display_csv_contents(csv_file_path)


In [ ]:
import pandas as pd
import crc16

def add_crc16_hash_column(df, input_column, output_column):
    df[output_column] = df[input_column].apply(lambda x: crc16.crc16xmodem(x.encode()))

# Load the CSV file into a pandas DataFrame
csv_file_path = "teams.csv"
df = pd.read_csv(csv_file_path)

# Call the function to add the "NAME_HASH" column
add_crc16_hash_column(df, input_column="TAG", output_column="NAME_HASH")

# Save the updated DataFrame to the CSV file
df.to_csv(csv_file_path, index=False)

# Display the updated DataFrame
print("Updated CSV File Contents:")
print(df)


In [ ]:
import pandas as pd

def find_duplicate_hashes(df):
    # Find duplicate values in the "NAME_HASH" column
    duplicates = df[df.duplicated(subset="NAME_HASH", keep=False)]

    # Output the IDs of rows with duplicate hashes
    if not duplicates.empty:
        print("Rows with Duplicate Hashes:")
        print(duplicates[["ID", "NAME_HASH"]])
    else:
        print("No Duplicate Hashes Found.")

# Load the CSV file into a pandas DataFrame
csv_file_path = "teams.csv"
df = pd.read_csv(csv_file_path)

# Call the function to find and display rows with duplicate hashes
find_duplicate_hashes(df)


In [ ]:
import pandas as pd
import random

def generate_unique_numbers(num_numbers, num_digits):
    # Generate unique random numbers with the specified number of digits
    random_numbers = set()
    while len(random_numbers) < num_numbers:
        random_numbers.add(str(random.randint(10**(num_digits-1), 10**num_digits - 1)))

    return list(random_numbers)

def create_challenge_salt_csv(file_path, num_numbers, num_digits):
    # Generate unique random numbers
    random_numbers = generate_unique_numbers(num_numbers, num_digits)

    # Create a DataFrame
    df = pd.DataFrame({"RandomNumber": random_numbers})

    # Save the DataFrame to the CSV file
    df.to_csv(file_path, index=False)

# Specify the path for the challenge_salt.csv file
challenge_salt_file_path = "challenges_salt.csv"

# Call the function to create the CSV file
create_challenge_salt_csv(challenge_salt_file_path, num_numbers=20, num_digits=7)

print(f"CSV File 'challenge_salt.csv' created successfully.")


In [24]:
import pandas as pd
import binascii
import random

def generate_unique_crc32_hashes(num_hashes):
    # Generate unique random CRC32 hashes
    unique_hashes = set()
    while len(unique_hashes) < num_hashes:
        hash_value = binascii.crc32(str(random.random()).encode())
        unique_hashes.add(hash_value)
    
    return list(unique_hashes)

def create_challenge_1_hashes(df_teams):
    # Generate unique CRC32 hashes
    unique_hashes = generate_unique_crc32_hashes(len(df_teams))

    # Create the "CHALLENGE_1_hash" column
    df_teams["CHALLENGE_1_hash"] = unique_hashes

# Load teams.csv into a DataFrame
teams_file_path = "teams.csv"
df_teams = pd.read_csv(teams_file_path)

# Call the function to create CHALLENGE_1 hashes
create_challenge_1_hashes(df_teams)

# Save the updated DataFrame back to the original CSV file
df_teams.to_csv(teams_file_path, index=False)

# Display the updated DataFrame with CHALLENGE_1 hashes
print("Updated DataFrame with CHALLENGE_1 Hashes:")
print(df_teams)


Updated DataFrame with CHALLENGE_1 Hashes:
    ID             TAG  NAME_HASH  CHALLENGE_1_hash
0    0   team_number_0      24863         808385152
1    1   team_number_1      28990        1188217473
2    2   team_number_2      16733        1168999047
3    3   team_number_3      20860         945798535
4    4   team_number_4       8603         584083209
5    5   team_number_5      12730        3465006604
6    6   team_number_6        473        3888804236
7    7   team_number_7       4600          68080781
8    8   team_number_8      57367        4169665551
9    9   team_number_9      61494        1057597199
10  10  team_number_10      26341        4239222417
11  11  team_number_11      30404         552895123
12  12  team_number_12      18087        3363574551
13  13  team_number_13      22150        1083042969
14  14  team_number_14       9825        3054550682
15  15  team_number_15      13888         114316287
16  16  team_number_16       1571         611662748
17  17  team_number_1

In [38]:
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont

def create_images(df_teams, output_folder="Images"):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Specify the path to the custom font file
    font_path = "B612Mono-Regular.ttf"  # Update with the path to your font file

    for index, row in df_teams.iterrows():
        team_name = f"Team_{index}"
        hash_value = str(row["CHALLENGE_1_HASH"])

        # Create an image with the hash text
        image_width = len(hash_value) * 62 # Adjust the multiplier as needed
        image_height = 200  # Adjust the height as needed
        image = Image.new("RGB", (image_width, image_height), color="black")

        draw = ImageDraw.Draw(image)

        # Load the custom font
        font_size = 90  # Adjust the size as needed
        font = ImageFont.truetype(font_path, size=font_size)

        # Position and draw the text on the image
        text_position = (10, 70)  # Adjust the position as needed
        draw.text(text_position, hash_value, fill="white", font=font)

        # Convert the image to grayscale
        image = image.convert('L')

        # Save the image in the output folder with a unique name for each team
        image_path = os.path.join(output_folder, f"{team_name}_hash_image.jpg")
        image.save(image_path)

# Load teams.csv into a DataFrame
teams_file_path = "teams.csv"
df_teams = pd.read_csv(teams_file_path)

# Call the function to create images based on the "CHALLENGE_1" column
create_images(df_teams)

print("Images generated successfully in the 'Images' folder.")


Images generated successfully in the 'Images' folder.
